# Full Data Load to Neo4j

This notebook loads the **complete** Aircraft Digital Twin dataset into Neo4j using the Python driver.

## What's Included

**Nodes:**
- Aircraft, System, Component, Sensor
- Airport, Flight, Delay
- MaintenanceEvent, Removal

**Relationships:**
- HAS_SYSTEM, HAS_COMPONENT, HAS_SENSOR
- OPERATES_FLIGHT, DEPARTS_FROM, ARRIVES_AT, HAS_DELAY
- HAS_EVENT, AFFECTS_SYSTEM, AFFECTS_AIRCRAFT
- HAS_REMOVAL, REMOVED_COMPONENT

## Prerequisites
- Neo4j Aura credentials from Lab 1
- CSV data files uploaded to Unity Catalog Volume
- `neo4j` Python package installed on cluster

## Instructions
1. Enter your Neo4j credentials in the Configuration cell below
2. Set `CLEAR_DATABASE = True` if you want to start fresh
3. Run all cells in order

## Section 1: Configuration

Enter your Neo4j Aura connection details below.

In [ ]:
# ============================================
# CONFIGURATION - Enter your Neo4j credentials
# ============================================

NEO4J_URI = ""  # e.g., "neo4j+s://xxxxxxxx.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""  # Your password from Lab 1

# Set to True to clear the database before loading
CLEAR_DATABASE = True

# Unity Catalog Volume path (pre-configured by workshop admin)
DATA_PATH = "/Volumes/aws-databricks-neo4j-lab/lab-schema/lab-volume"

# Batch size for transaction handling
BATCH_SIZE = 1000

# Validate configuration
if not NEO4J_URI or not NEO4J_PASSWORD:
    print("WARNING: Please enter your Neo4j credentials above before running!")
else:
    print("Configuration ready!")
    print(f"Neo4j URI: {NEO4J_URI}")
    print(f"Data Path: {DATA_PATH}")
    print(f"Clear Database: {CLEAR_DATABASE}")

## Section 2: Connect to Neo4j

In [ ]:
import csv
import time
from typing import Any

from neo4j import GraphDatabase

# Connect to Neo4j
print(f"Connecting to Neo4j at {NEO4J_URI}...")
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Verify connection
start = time.time()
driver.verify_connectivity()
elapsed = (time.time() - start) * 1000
print(f"[OK] Connected in {elapsed:.0f}ms")

## Section 3: Helper Functions

In [ ]:
def read_csv(filename: str) -> list[dict[str, Any]]:
    """Read CSV file from UC Volume and return list of dictionaries."""
    full_path = f"{DATA_PATH}/{filename}"
    with open(full_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        return list(reader)


def run_in_batches(records: list[dict], query: str, batch_size: int = BATCH_SIZE):
    """Execute a query for records in batches."""
    total = len(records)
    for i in range(0, total, batch_size):
        batch = records[i : i + batch_size]
        driver.execute_query(query, batch=batch)
        progress = min(i + batch_size, total)
        print(f"  Progress: {progress}/{total} ({100 * progress // total}%)", end="\r")
    print()


print("Helper functions defined.")

## Section 4: Clear Database (Optional)

In [ ]:
if CLEAR_DATABASE:
    print("Clearing database...")
    deleted_total = 0
    while True:
        records, _, _ = driver.execute_query(
            "MATCH (n) WITH n LIMIT 500 DETACH DELETE n RETURN count(*) AS deleted"
        )
        count = records[0]["deleted"]
        deleted_total += count
        if count > 0:
            print(f"  Deleted {deleted_total} nodes so far...", end="\r")
        if count == 0:
            break
    print(f"\n[OK] Database cleared ({deleted_total} nodes deleted).")
else:
    print("Skipping database clear (CLEAR_DATABASE = False)")

## Section 5: Create Constraints and Indexes

In [ ]:
print("Creating constraints...")
constraints = [
    ("Aircraft", "aircraft_id"),
    ("System", "system_id"),
    ("Component", "component_id"),
    ("Sensor", "sensor_id"),
    ("Airport", "airport_id"),
    ("Flight", "flight_id"),
    ("Delay", "delay_id"),
    ("MaintenanceEvent", "event_id"),
    ("Removal", "removal_id"),
]

for label, prop in constraints:
    try:
        driver.execute_query(
            f"CREATE CONSTRAINT IF NOT EXISTS FOR (n:{label}) REQUIRE n.{prop} IS UNIQUE"
        )
        print(f"  [OK] Constraint: {label}.{prop}")
    except Exception as e:
        print(f"  [WARN] {label}.{prop}: {e}")

print("\nCreating indexes...")
indexes = [
    ("MaintenanceEvent", "severity"),
    ("Flight", "aircraft_id"),
    ("Removal", "aircraft_id"),
]

for label, prop in indexes:
    try:
        index_name = f"idx_{label.lower()}_{prop.lower()}"
        driver.execute_query(f"CREATE INDEX {index_name} IF NOT EXISTS FOR (n:{label}) ON (n.{prop})")
        print(f"  [OK] Index: {label}.{prop}")
    except Exception as e:
        print(f"  [WARN] {label}.{prop}: {e}")

## Section 6: Load Nodes

Load all node types from CSV files.

In [ ]:
print("Loading Aircraft nodes...")
records = read_csv("nodes_aircraft.csv")
query = """
UNWIND $batch AS row
MERGE (a:Aircraft {aircraft_id: row[':ID(Aircraft)']})
SET a.tail_number = row['tail_number'],
    a.icao24 = row['icao24'],
    a.model = row['model'],
    a.manufacturer = row['manufacturer'],
    a.operator = row['operator']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} aircraft.")

In [ ]:
print("Loading System nodes...")
records = read_csv("nodes_systems.csv")
query = """
UNWIND $batch AS row
MERGE (s:System {system_id: row[':ID(System)']})
SET s.aircraft_id = row['aircraft_id'],
    s.type = row['type'],
    s.name = row['name']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} systems.")

In [ ]:
print("Loading Component nodes...")
records = read_csv("nodes_components.csv")
query = """
UNWIND $batch AS row
MERGE (c:Component {component_id: row[':ID(Component)']})
SET c.system_id = row['system_id'],
    c.type = row['type'],
    c.name = row['name']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} components.")

In [ ]:
print("Loading Sensor nodes...")
records = read_csv("nodes_sensors.csv")
query = """
UNWIND $batch AS row
MERGE (s:Sensor {sensor_id: row[':ID(Sensor)']})
SET s.system_id = row['system_id'],
    s.type = row['type'],
    s.name = row['name'],
    s.unit = row['unit']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} sensors.")

In [ ]:
print("Loading Airport nodes...")
records = read_csv("nodes_airports.csv")
query = """
UNWIND $batch AS row
MERGE (a:Airport {airport_id: row[':ID(Airport)']})
SET a.name = row['name'],
    a.city = row['city'],
    a.country = row['country'],
    a.iata = row['iata'],
    a.icao = row['icao'],
    a.lat = toFloat(row['lat']),
    a.lon = toFloat(row['lon'])
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} airports.")

In [ ]:
print("Loading Flight nodes...")
records = read_csv("nodes_flights.csv")
query = """
UNWIND $batch AS row
MERGE (f:Flight {flight_id: row[':ID(Flight)']})
SET f.flight_number = row['flight_number'],
    f.aircraft_id = row['aircraft_id'],
    f.operator = row['operator'],
    f.origin = row['origin'],
    f.destination = row['destination'],
    f.scheduled_departure = row['scheduled_departure'],
    f.scheduled_arrival = row['scheduled_arrival']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} flights.")

In [ ]:
print("Loading Delay nodes...")
records = read_csv("nodes_delays.csv")
query = """
UNWIND $batch AS row
MERGE (d:Delay {delay_id: row[':ID(Delay)']})
SET d.cause = row['cause'],
    d.minutes = toInteger(row['minutes'])
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} delays.")

In [ ]:
print("Loading MaintenanceEvent nodes...")
records = read_csv("nodes_maintenance.csv")
query = """
UNWIND $batch AS row
MERGE (m:MaintenanceEvent {event_id: row[':ID(MaintenanceEvent)']})
SET m.component_id = row['component_id'],
    m.system_id = row['system_id'],
    m.aircraft_id = row['aircraft_id'],
    m.fault = row['fault'],
    m.severity = row['severity'],
    m.reported_at = row['reported_at'],
    m.corrective_action = row['corrective_action']
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} maintenance events.")

In [ ]:
print("Loading Removal nodes...")
records = read_csv("nodes_removals.csv")
query = """
UNWIND $batch AS row
MERGE (r:Removal {removal_id: row[':ID(RemovalEvent)']})
SET r.component_id = row['component_id'],
    r.aircraft_id = row['aircraft_id'],
    r.removal_date = row['removal_date'],
    r.reason = row['RMV_REA_TX'],
    r.tsn = toFloat(row['time_since_install']),
    r.csn = toInteger(row['flight_cycles_at_removal'])
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} removals.")

## Section 7: Load Relationships

In [ ]:
print("Loading HAS_SYSTEM relationships...")
records = read_csv("rels_aircraft_system.csv")
query = """
UNWIND $batch AS row
MATCH (a:Aircraft {aircraft_id: row[':START_ID(Aircraft)']})
MATCH (s:System {system_id: row[':END_ID(System)']})
MERGE (a)-[:HAS_SYSTEM]->(s)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_SYSTEM relationships.")

In [ ]:
print("Loading HAS_COMPONENT relationships...")
records = read_csv("rels_system_component.csv")
query = """
UNWIND $batch AS row
MATCH (s:System {system_id: row[':START_ID(System)']})
MATCH (c:Component {component_id: row[':END_ID(Component)']})
MERGE (s)-[:HAS_COMPONENT]->(c)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_COMPONENT relationships.")

In [ ]:
print("Loading HAS_SENSOR relationships...")
records = read_csv("rels_system_sensor.csv")
query = """
UNWIND $batch AS row
MATCH (s:System {system_id: row[':START_ID(System)']})
MATCH (sn:Sensor {sensor_id: row[':END_ID(Sensor)']})
MERGE (s)-[:HAS_SENSOR]->(sn)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_SENSOR relationships.")

In [ ]:
print("Loading HAS_EVENT relationships...")
records = read_csv("rels_component_event.csv")
query = """
UNWIND $batch AS row
MATCH (c:Component {component_id: row[':START_ID(Component)']})
MATCH (m:MaintenanceEvent {event_id: row[':END_ID(MaintenanceEvent)']})
MERGE (c)-[:HAS_EVENT]->(m)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_EVENT relationships.")

In [ ]:
print("Loading OPERATES_FLIGHT relationships...")
records = read_csv("rels_aircraft_flight.csv")
query = """
UNWIND $batch AS row
MATCH (a:Aircraft {aircraft_id: row[':START_ID(Aircraft)']})
MATCH (f:Flight {flight_id: row[':END_ID(Flight)']})
MERGE (a)-[:OPERATES_FLIGHT]->(f)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} OPERATES_FLIGHT relationships.")

In [ ]:
print("Loading DEPARTS_FROM relationships...")
records = read_csv("rels_flight_departure.csv")
query = """
UNWIND $batch AS row
MATCH (f:Flight {flight_id: row[':START_ID(Flight)']})
MATCH (a:Airport {airport_id: row[':END_ID(Airport)']})
MERGE (f)-[:DEPARTS_FROM]->(a)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} DEPARTS_FROM relationships.")

In [ ]:
print("Loading ARRIVES_AT relationships...")
records = read_csv("rels_flight_arrival.csv")
query = """
UNWIND $batch AS row
MATCH (f:Flight {flight_id: row[':START_ID(Flight)']})
MATCH (a:Airport {airport_id: row[':END_ID(Airport)']})
MERGE (f)-[:ARRIVES_AT]->(a)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} ARRIVES_AT relationships.")

In [ ]:
print("Loading HAS_DELAY relationships...")
records = read_csv("rels_flight_delay.csv")
query = """
UNWIND $batch AS row
MATCH (f:Flight {flight_id: row[':START_ID(Flight)']})
MATCH (d:Delay {delay_id: row[':END_ID(Delay)']})
MERGE (f)-[:HAS_DELAY]->(d)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_DELAY relationships.")

In [ ]:
print("Loading AFFECTS_SYSTEM relationships...")
records = read_csv("rels_event_system.csv")
query = """
UNWIND $batch AS row
MATCH (m:MaintenanceEvent {event_id: row[':START_ID(MaintenanceEvent)']})
MATCH (s:System {system_id: row[':END_ID(System)']})
MERGE (m)-[:AFFECTS_SYSTEM]->(s)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} AFFECTS_SYSTEM relationships.")

In [ ]:
print("Loading AFFECTS_AIRCRAFT relationships...")
records = read_csv("rels_event_aircraft.csv")
query = """
UNWIND $batch AS row
MATCH (m:MaintenanceEvent {event_id: row[':START_ID(MaintenanceEvent)']})
MATCH (a:Aircraft {aircraft_id: row[':END_ID(Aircraft)']})
MERGE (m)-[:AFFECTS_AIRCRAFT]->(a)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} AFFECTS_AIRCRAFT relationships.")

In [ ]:
print("Loading HAS_REMOVAL relationships...")
records = read_csv("rels_aircraft_removal.csv")
query = """
UNWIND $batch AS row
MATCH (a:Aircraft {aircraft_id: row[':START_ID(Aircraft)']})
MATCH (r:Removal {removal_id: row[':END_ID(RemovalEvent)']})
MERGE (a)-[:HAS_REMOVAL]->(r)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} HAS_REMOVAL relationships.")

In [ ]:
print("Loading REMOVED_COMPONENT relationships...")
records = read_csv("rels_component_removal.csv")
query = """
UNWIND $batch AS row
MATCH (r:Removal {removal_id: row[':END_ID(RemovalEvent)']})
MATCH (c:Component {component_id: row[':START_ID(Component)']})
MERGE (r)-[:REMOVED_COMPONENT]->(c)
"""
run_in_batches(records, query)
print(f"[OK] Loaded {len(records)} REMOVED_COMPONENT relationships.")

## Section 8: Summary

In [ ]:
print("=" * 60)
print("LOAD COMPLETE - Summary")
print("=" * 60)

# Count nodes
node_counts, _, _ = driver.execute_query("""
    CALL () {
        MATCH (n:Aircraft) RETURN 'Aircraft' as label, count(n) as count
        UNION ALL
        MATCH (n:System) RETURN 'System' as label, count(n) as count
        UNION ALL
        MATCH (n:Component) RETURN 'Component' as label, count(n) as count
        UNION ALL
        MATCH (n:Sensor) RETURN 'Sensor' as label, count(n) as count
        UNION ALL
        MATCH (n:Airport) RETURN 'Airport' as label, count(n) as count
        UNION ALL
        MATCH (n:Flight) RETURN 'Flight' as label, count(n) as count
        UNION ALL
        MATCH (n:Delay) RETURN 'Delay' as label, count(n) as count
        UNION ALL
        MATCH (n:MaintenanceEvent) RETURN 'MaintenanceEvent' as label, count(n) as count
        UNION ALL
        MATCH (n:Removal) RETURN 'Removal' as label, count(n) as count
    }
    RETURN label, count
    ORDER BY count DESC
""")

print("\nNode Counts:")
total_nodes = 0
for row in node_counts:
    print(f"  {row['label']}: {row['count']:,}")
    total_nodes += row['count']
print(f"  ---------------------")
print(f"  Total Nodes: {total_nodes:,}")

# Count relationships
rel_records, _, _ = driver.execute_query("MATCH ()-[r]->() RETURN count(r) as count")
rel_count = rel_records[0]["count"]
print(f"\nTotal Relationships: {rel_count:,}")

print("\n" + "=" * 60)

In [ ]:
# Close the driver connection
driver.close()
print("Neo4j driver closed.")

## Next Steps

Your Neo4j database now contains the complete Aircraft Digital Twin dataset!

### Explore in Neo4j Aura

1. Go to [console.neo4j.io](https://console.neo4j.io)
2. Open your instance and click **Query**

### Sample Queries to Try

**View the complete schema:**
```cypher
CALL db.schema.visualization()
```

**Find aircraft with critical maintenance issues:**
```cypher
MATCH (a:Aircraft)-[:HAS_SYSTEM]->(s:System)-[:HAS_COMPONENT]->(c:Component)-[:HAS_EVENT]->(m:MaintenanceEvent)
WHERE m.severity = 'CRITICAL' AND m.reported_at IS NOT NULL
RETURN a.tail_number, s.name, c.name, m.fault, m.reported_at
ORDER BY m.reported_at DESC
LIMIT 10
```

**Analyze flight delays by cause:**
```cypher
MATCH (f:Flight)-[:HAS_DELAY]->(d:Delay)
RETURN d.cause, count(*) AS count, avg(d.minutes) AS avg_minutes
ORDER BY count DESC
```

**Find component removal history:**
```cypher
MATCH (a:Aircraft)-[:HAS_REMOVAL]->(r:Removal)-[:REMOVED_COMPONENT]->(c:Component)
WHERE r.removal_date IS NOT NULL
RETURN a.tail_number, c.name, r.reason, r.removal_date, r.tsn, r.csn
ORDER BY r.removal_date DESC
LIMIT 20
```